# Kaggle Wildfires Data

Cleaning, preprocessing, feature engineering, and then saving to .csv

This is part of a larger project analyzing California wildfires.

In [1]:
import numpy as np
import pandas as pd
import datetime
import sqlite3

In [2]:
# Filepath for the different files we want to combine
filepath = ''
sqlite_file = 'FPA_FOD_20170508.sqlite'

### Extract data from SQL table

Open SQLite database, extract the wildfires table, save to pandas dataframe, and close the database

In [3]:
# Connect to SQLite database
conn = sqlite3.connect(filepath+sqlite_file)

# Read files into dataframe
df = pd.read_sql_query("select * from fires", conn)

# Close the connection
conn.close()

View the columns in the dataframe

In [4]:
df.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')

In [5]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


### Subset data and drop unneeded columns

Subset to only fires in CA between 2000-2015

In [6]:
CAfires = df[df.STATE == 'CA']
CAfires = CAfires[CAfires.FIRE_YEAR > 1999]

Drop columns we do not need in the analysis

In [7]:
CAfires = CAfires.drop(['FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'STAT_CAUSE_CODE', 'OWNER_CODE', 'Shape', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'CONT_DOY', 'CONT_TIME'], axis=1)
CAfires.columns

Index(['OBJECTID', 'FIRE_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE',
       'STAT_CAUSE_DESCR', 'CONT_DATE', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE',
       'FIPS_NAME'],
      dtype='object')

### Fix date format

The comments in Kaggle about this dataset say the dates are in Julian format.  We will fix this.

In [8]:
CAfires['DISCOVERY_DATE'].sort_values()

1097337    2451544.5
1068757    2451544.5
363556     2451544.5
1073745    2451544.5
1067119    2451544.5
             ...    
1872147    2457386.5
1871985    2457387.5
1872288    2457387.5
1871977    2457387.5
1872285    2457387.5
Name: DISCOVERY_DATE, Length: 121535, dtype: float64

Convert dates to Julian Dates

In [9]:
epoch = pd.to_datetime(0, unit='s').to_julian_date()

Convert Julian dates to standard dates for fire discovery date

In [10]:
CAfires['DISCOVERY_DATE'] = CAfires['DISCOVERY_DATE'] - epoch
CAfires['DISCOVERY_DATE'] = pd.to_datetime(CAfires['DISCOVERY_DATE'], unit='D')

Check for missing values in this column

In [11]:
# Discovery Date
CAfires['DISCOVERY_DATE'].isnull().sum()

0

Convert Julian dates to standard dates for fire containment date

In [12]:
CAfires['CONT_DATE'] = CAfires['CONT_DATE'] - epoch
CAfires['CONT_DATE'] = pd.to_datetime(CAfires['CONT_DATE'], unit='D')

Check for missing values in this column

In [13]:
CAfires['CONT_DATE'].isnull().sum()

56162

Drop rows where no containment date is available

In [14]:
CAfires = CAfires[CAfires['CONT_DATE'].notnull()]

In [15]:
# How many rows of data remain?
len(CAfires)

65373

### Feature engineer new variables

Create column for fire length

In [16]:
CAfires['Fire Length'] = CAfires['CONT_DATE'] - CAfires['DISCOVERY_DATE']

Descriptive statistics on fire length

In [17]:
CAfires['Fire Length'].max()

Timedelta('1881 days 00:00:00')

In [18]:
CAfires['Fire Length'].min()

Timedelta('0 days 00:00:00')

In [19]:
CAfires['Fire Length'].mean()

Timedelta('1 days 01:18:30.348308')

In [20]:
CAfires['Fire Length'].sort_values()

0            0 days
1567892      0 days
1567893      0 days
1567894      0 days
1567895      0 days
             ...   
1063028    371 days
298607     374 days
368029    1461 days
365708    1826 days
356156    1881 days
Name: Fire Length, Length: 65373, dtype: timedelta64[ns]

One of the longest burning fires in recorded history lasted about 6 months. We will delete rows where the fire length excedes 180 days since the data is likely wrong.

In [21]:
CAfires['length'] = CAfires['Fire Length'].dt.days
CAfires = CAfires[CAfires['length'] < 180]

It makes more sense for the minimum length of a fire to be one day rather than zero days, even if the fire starts and is contained on the same day.

In [22]:
CAfires.loc[CAfires['length']<1, 'length'] = 1
CAfires['Fire Length'] = pd.to_timedelta(CAfires['length'], unit='d')

Let's look to see if there are patterns of when fires start/are discovered based on day of the week

In [23]:
# Create discovery day of week variable
CAfires['DISCOVERY_DOW'] = CAfires['DISCOVERY_DATE'].dt.dayofweek
CAfires['DISCOVERY_DOW'].value_counts().sort_values()

3     8737
4     8757
1     8851
0     9152
2     9266
5    10240
6    10358
Name: DISCOVERY_DOW, dtype: int64

Weekends seem to have more fires than work days.  Let's create variables for this.

In [24]:
# Create variable for weekend
CAfires['DISCOVERY_WE'] = CAfires['DISCOVERY_DOW'].apply(lambda x: 0 if x<5 else 1)

# Create variable for workday
CAfires['DISCOVERY_WORKDAY'] = CAfires['DISCOVERY_DOW'].apply(lambda x: 1 if x<5 else 0)

### Rather than reflect one row for each fire, we will expand the dates to reflect the fire for each day during which it burned.

In [25]:
df2 = pd.DataFrame(np.repeat(CAfires.values, CAfires['Fire Length'].dt.days, axis=0), columns=CAfires.columns)
df2['BurningDay'] = df2.groupby('OBJECTID')['DISCOVERY_DATE'].transform(lambda x: pd.date_range(start=x.iloc[0], periods = len(x)))

In [26]:
df2.head(20)

,OBJECTID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,STAT_CAUSE_DESCR,CONT_DATE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,...,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Fire Length,length,DISCOVERY_DOW,DISCOVERY_WE,DISCOVERY_WORKDAY,BurningDay
0,1,FOUNTAIN,2005,2005-02-02,Miscellaneous,2005-02-02,0.1,A,40.0369,-121.006,...,CA,63,063,Plumas,1 days,1,2,0,1,2005-02-02
1,2,PIGEON,2004,2004-05-12,Lightning,2004-05-12,0.25,A,38.9331,-120.404,...,CA,61,061,Placer,1 days,1,2,0,1,2004-05-12
2,3,SLACK,2004,2004-05-31,Debris Burning,2004-05-31,0.1,A,38.9842,-120.736,...,CA,17,017,El Dorado,1 days,1,0,0,1,2004-05-31
3,4,DEER,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.913,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-06-28
4,4,DEER,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.913,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-06-29
5,4,DEER,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.913,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-06-30
6,4,DEER,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.913,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-07-01
7,4,DEER,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.913,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-07-02
8,5,STEVENOT,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.933,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-06-28
9,5,STEVENOT,2004,2004-06-28,Lightning,2004-07-03,0.1,A,38.5592,-119.933,...,CA,3,003,Alpine,5 days,5,0,0,1,2004-06-29


### Write cleaned up dataframe to .csv file

In [27]:
df2.to_csv(filepath + 'KaggleCleaned.csv', index=False)